In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sqlalchemy import create_engine # database connection

disk_engine_ib = create_engine('sqlite:///ehr_ib.db')

#train = pd.read_csv('./input/train.csv', iterator=True, chunksize=100000)
#test = pd.read_csv('./input/test.csv', iterator=True, chunksize=100000)
#destinations = pd.read_csv('./input/destinations.csv', iterator=True, chunksize=15)

#chunk = train.get_chunk()
#test_chunk = test.get_chunk()
train = pd.read_sql_query('SELECT * FROM train_ib1_cnt0 LIMIT 600000', disk_engine_ib)
train = train.drop(['level_0', 'index', 'user_id', 'is_mobile'], axis=1)
train = train.fillna(0)

train.head()

,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,hotel_continent,hotel_country,hotel_market,hotel_cluster
0,2,3,66,348,48862,2234.2641,1,9,18,22,2,0,1,8250,1,2,50,628,1
1,2,3,66,318,52078,0.0000,1,4,50,52,2,0,1,8291,1,2,50,191,2
2,30,4,195,548,56440,0.0000,1,9,83,91,2,0,1,1385,1,0,185,185,58
3,30,4,195,991,47725,0.0000,0,9,215,217,2,0,1,8803,1,3,151,69,36
4,2,3,66,462,41898,2454.8588,1,1,50,55,3,0,2,12009,1,2,50,680,95


In [6]:
xx = train.sample(1000)
xx.head()

,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,hotel_continent,hotel_country,hotel_market,hotel_cluster
209612,11,3,205,135,36086,442.1929,1,9,23,27,2,0,1,12270,6,2,50,663,70
131669,2,3,202,269,25293,0.0000,0,9,2,4,2,0,1,1549,3,3,5,50,36
370616,11,3,66,346,28279,231.8497,0,9,0,1,2,0,1,23297,6,2,50,631,3
491399,2,3,66,174,33085,0.0000,0,9,1,2,1,0,1,12271,6,2,50,663,72
328763,2,3,66,174,41886,194.8685,0,5,23,28,2,0,1,8250,1,2,50,628,47


In [2]:

X = train.drop(['hotel_cluster'], axis=1)
Y = train['hotel_cluster']

X = X.as_matrix()
Y = Y.as_matrix()

In [3]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import log_loss
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)


clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.3,\
                                 max_depth=1, verbose=1, \
                                max_leaf_nodes= 5, subsample= 0.8)

clf.fit(X_train, y_train)
clf.score(X_test, y_test)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1 722252613445140864526260254979761312223770058907304812802671702467767465372128207849046257713751216010199674612105466430810728010494200598933578983872024637733896978432.0000 -188078000141117227762863547404979777111399793785874172129794665307351612497709179232461980749766682468648600660488242485163460334088475785953343523776504587550489313280.0000          407.87m
         2 729836403773411759131568058987637091133985503460002633010840482577312706227185536791805837070141391505769498551025847731810105949352154924932210441897008129755806957568.0000           0.0000          410.18m
         3 724734581188938661369106186647612218975728345498414911129821163373253268744663665611827912933828540042944649994884113939096364320120059826874175573346510514565339414528.0000           0.0000          413.13m
         4 726113452157715063867299161485860731006048073573337485520116291219084001168382350147633354240025405456652035476

KeyboardInterrupt: 

In [89]:
# Extraction of latents for [#'hotel_market', 'srch_destination_id', 'user_id',
# 'user_location_city', 'user_location_region']
def target_extractor(chunk):
    Y_df = pd.DataFrame(data=chunk['hotel_cluster'])
    Y = Y_df.as_matrix()

    #enc = OneHotEncoder(n_values=100, dtype= 'int', sparse=False)
    #Y = enc.fit_transform(Y_df.as_matrix())
    
    return Y

#chnk = chnk.drop('hotel_cluster', axis=1)
#Y

In [ ]:
def feature_extractor(chunk):
    from sklearn.preprocessing import OneHotEncoder
    import pandas as pd

    categorical_columns = ['posa_continent', 'hotel_continent', 'srch_destination_type_id',
                      'channel', 'site_name', 'hotel_country', 'user_location_country']
    n_values_categorical_columns =[5, 7, 10, 11, 54, 213, 240]

    latent_columns = ['user_location_region', 'hotel_market', 'user_location_city', 
                  'srch_destination_id', 'user_id']
    # n_values_latent_columns = [1027, 2117, 56508, 65107, 1198785]

    other_colums = ['is_booking', 'is_mobile', 'is_package', 'srch_rm_cnt', 'srch_children_cnt'
                'srch_adults_cnt', 'cnt', 'orig_destination_distance']

    enc = OneHotEncoder(n_values = n_values_categorical_columns, dtype= 'int', sparse=False)
    X_df = pd.DataFrame(data= enc.fit_transform(chunk[categorical_columns].as_matrix()))

    X_df['date_time'] = pd.DataFrame(data=pd.to_datetime(chunk['date_time']))
    X_df['month'] = X_df['date_time'].map(lambda x: x.month)

    X_df['ci'] = pd.to_datetime(chunk['srch_ci'])
    X_df['co'] = pd.to_datetime(chunk['srch_co'])

    X_df['ci'] = (X_df['ci'] - X_df['date_time']).fillna(0)
    X_df['ci'] = X_df['ci'].map(lambda x: x.days)

    X_df['co'] = (X_df['co'] - X_df['date_time']).fillna(0)
    X_df['co'] = X_df['co'].map(lambda x: x.days)

    X_df = X_df.drop('date_time',axis=1)

    return X_df.as_matrix()


In [94]:
from sklearn import linear_model
from sklearn import metrics
chunk = train.get_chunk()

clf = linear_model.SGDClassifier(loss='log')

#for chunk in train:
Y = target_extractor(chunk)
X = feature_extractor(chunk)
#X = X_df.as_matrix()
clf.partial_fit(X, Y, classes=np.arange(1, 100))
Y_prob = clf.predict_proba(X)
#clf.score(X, Y)
#Y_pred = clf.predict(X)
#X

C:\Users\Dariyush\Anaconda3\lib\site-packages\sklearn\utils\validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [95]:
Y

array([[ 2],
       [ 2],
       [90],
       ..., 
       [98],
       [98],
       [98]], dtype=int64)

In [127]:
np.argmax(Y_prob,axis=1)

array([89, 55, 38, ..., 24, 58, 58], dtype=int64)

In [ ]:
df_max=pd.DataFrame()
df_min=pd.DataFrame()
chunk_shape = []
for chunk in train:
    df_max = df_max.append(chunk.max(),ignore_index=True)
    df_min = df_min.append(chunk.min(),ignore_index=True)
    chunk_shape.append(chunk.shape)


In [ ]:
data_max = df_max.max().astype(int)

data_max

In [ ]:
data_max[['site_name', 'posa_continent', 'user_location_country',
       'user_location_region', 'user_location_city', 'user_id', 'is_mobile', 'is_package',
       'channel', 'srch_destination_id', 'srch_destination_type_id',
       'is_booking', 'hotel_continent', 'hotel_country', 'hotel_market', 'hotel_cluster']]

In [ ]:
'user_location_city', 'user_id', 'srch_destination_id', 'hotel_market'

In [ ]:
sample_submission.get_chunk()